In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import mediapipe as mp
import os

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score


### keypoints using MP holistic

In [6]:
mp_holistic = mp.solutions.holistic # Modelo Holístico
mp_desenho = mp.solutions.drawing_utils # Utilidades para desenhos.

In [7]:
def mediapipe_detection(imagem, modelo):
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB) # Conversão de cor, de BGR to RGB.
    imagem.flags.writeable = False                   # Desabilita a escrita na imagem
    results = modelo.process(imagem)                 # Faz as previsões.
    imagem.flags.writeable = True                    # Abilita a escrita na imagem.
    imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2BGR) # Conversão de cor, de RGB to BGR.
    return imagem, results

In [8]:
def desenhar_pontos(imagem, resultados):
    mp_desenho.draw_landmarks(imagem, resultados.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                              mp_desenho.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1), # Cor/Estilo dos pontos.
                              mp_desenho.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1) # Cor/Estilo das conexões.
                              ) # Desenhar as conexões e os pontos no contorno da FACE
    
    mp_desenho.draw_landmarks(imagem, resultados.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_desenho.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_desenho.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2),
                              ) # Desenhar as conexões e os pontos no Tronco
    
    mp_desenho.draw_landmarks(imagem, resultados.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_desenho.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_desenho.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              ) # Desenhar as conexões e os pontos na mão esquerda
    
    mp_desenho.draw_landmarks(imagem, resultados.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_desenho.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_desenho.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              ) # Desenhar as conexões e os pontos na mão direita
    return imagem


### Extract Keypoint Values

In [9]:
# Função para extrair as variáveis espaciais dos landmarks de cada frame:
def extract_points(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose, face, left_hand, right_hand])

### Setup Folders for Collection

In [12]:
DATA_PATH = os.path.join('MP_DATA_2')

# Classes/Ações que queremos detectar: (Podemos expandir isso!!)
actions = np.array([chr(i) for i in range(ord('A'), ord('I') + 1)])
 
number_of_videos = 30 # Número de sequências ou vídeos que iremos coletar os frames (Pode ser modificado!)
sequence_length = 180 # Para cada vídeo, iremos coletar 30 frames. (Pode ser modificado!)

In [13]:
for action in actions:
    for sequence in range(number_of_videos):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except FileExistsError:
            print("Folder já existe!!")

In [14]:
import os
import shutil

# Define the paths
MP_DATA = os.path.join('MP_DATA')
MP_DATA_2 = os.path.join('MP_DATA_2')
mp_final = os.path.join('MP_DATA_final')

# Ensure the final path exists
if not os.path.exists(mp_final):
    os.makedirs(mp_final)

# Iterate over each of the lettered folders (A to I)
for folder in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']:
    
    # Define the paths for the current lettered folder for MP_DATA and MP_DATA_2
    path1 = os.path.join(MP_DATA, folder)
    path2 = os.path.join(MP_DATA_2, folder)
    
    # Define the final path for the lettered folder and ensure it exists
    final_path = os.path.join(mp_final, folder)
    if not os.path.exists(final_path):
        os.makedirs(final_path)

    # First, copy the contents of the lettered folder from MP_DATA
    for subfolder in range(30):
        src = os.path.join(path1, str(subfolder))
        dest = os.path.join(final_path, str(subfolder))
        
        if os.path.exists(src):
            shutil.copytree(src, dest)

    # Next, copy the contents of the lettered folder from MP_DATA_2 and rename them from 30 to 59
    for subfolder in range(30):
        src = os.path.join(path2, str(subfolder))
        dest = os.path.join(final_path, str(subfolder + 30))
        
        if os.path.exists(src):
            shutil.copytree(src, dest)

### Collecting Keypoints Values for training and testing

In [15]:
DATA_PATH = 'MP_DATA_final'

cap = cv2.VideoCapture(0) # Acessa a nossa Webcam.

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: # Obre o modelo usando ooperador de contexto.

    for action in ['C']:
        for sequence in range(30, 32):
            flag = False
            for numero_frame in range(sequence_length + 1):
                
                # Lê o frame:
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
                    
                # Desenha os landmarks:
                desenhar_pontos(image, results)

                if cv2.waitKey(8) & 0xFF == ord('p'):
                    print("VIDEO PAUSED")
                    while True:
                        if cv2.waitKey(8) & 0xFF == ord('p'):
                            print("VIDEO PLAYED")
                            break

                # Lógica para organizar a coleta dos dados:
                if numero_frame == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Coletando os frames para {} - video numero {}'.format(action, sequence), (15, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)
                    cv2.putText(image, 'PRESS R TO START', (120, 250),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                else:
                    cv2.putText(image, 'Coletando os frames para {} - video numero {} - frame {}'.format(action, sequence, numero_frame), (15, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)

                if numero_frame != 0:
                    results_extraction = extract_points(results)
                    numpy_file_path = os.path.join(DATA_PATH, action, str(sequence), str(numero_frame-1))
                    np.save(numpy_file_path, results_extraction)

                # Mostra a tela:
                cv2.imshow('OpenCV Feed', image)

                if numero_frame == 0:
                    while flag == False:
                        if cv2.waitKey(8) & 0xFF == ord('r'):
                            # cv2.waitKey(2000)
                            flag = True
                
                # Sai do loop depois de 10 seg ou quando o usuário aperta a letra "q":
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

In [48]:
cap.release()
cv2.destroyAllWindows()

### Preprocess Data and Creating labels and Features

In [51]:
label_map = {label:num for num, label in enumerate(actions)}

In [52]:
sequences, labels = [], []

for action in actions:
    for sequence in range(number_of_videos):
        janela = []
        for numero_frame in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(numero_frame)))
            janela.append(res)
        sequences.append(janela)
        labels.append(label_map[action])

KeyboardInterrupt: 